### HW

* Осуществите фильтрацию изображения при помощи среднего и среднего взвешенного фильтра с различным размером ядра. 
* Сравните результаты медианного и среднего фильтра на изображении с шумом соли и перца.
* Реализуйте медианный фильтр. Сравните с изображением, которое получается в результате применения функции cv2.medianBlur().
* Реализуйте метод conv2D для выполнения операции свертки со следующими параметрами: img (входное изображение), kernel (ядро свертки), padding (отступ, по умолчанию 0), stride (шаг, по умолчанию 1). Сравните с изображением, которое получается в результате применения функции cv2.filter2D().



In [ ]:
import cv2 as cv
import PIL as pil
from matplotlib import pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
img = cv.imread('img.png')
img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
img_gray = cv.imread('img.png', cv.IMREAD_GRAYSCALE)
plt.imshow(img_gray, 'gray')

* Осуществите фильтрацию изображения при помощи среднего и среднего взвешенного фильтра с различным размером ядра. 

In [ ]:
def printImgs(imgs):
    fig = plt.figure(figsize=(8,24), dpi=500)
    img_len = len(imgs)
    for idx, img in enumerate(imgs):
        fig.add_subplot(img_len, 1, idx + 1)
        plt.imshow(img)

In [ ]:
printImgs([
    cv.blur(img,(3,3)),
    cv.blur(img,(5,5)),
    cv.blur(img,(7,7)),
    cv.blur(img,(9,9)),
])

In [ ]:
printImgs([
    cv.medianBlur(img, 3),
    cv.medianBlur(img, 5),
    cv.medianBlur(img, 7),
    cv.medianBlur(img, 9),
])

* Сравните результаты медианного и среднего фильтра на изображении с шумом соли и перца.

In [ ]:
def add_sp_noise(image, s_vs_p_coef=0.5, amount=0.04):
    if (len(image.shape) == 2):
        row,col = image.shape
    elif (len(image.shape) == 3):
        row,col,_ = image.shape

    out = np.copy(image)

    # Salt mode
    num_salt = int(np.ceil(amount * image.size * s_vs_p_coef))
    coords = [(np.random.randint(0, int(row - 1)), np.random.randint(0, int(col - 1)))
                for i in range(num_salt)]
    for coord in coords:
        out[coord[0]][coord[1]] = 255


    # Pepper mode
    num_pepper = int(np.ceil(amount* image.size * (1. - s_vs_p_coef)))
    coords = [(np.random.randint(0, int(row - 1)), np.random.randint(0, int(col - 1)))
                for i in range(num_pepper)]
    for coord in coords:
        out[coord[0]][coord[1]] = 0

    return out

In [ ]:
plt.imshow(add_sp_noise(img))

In [ ]:
sp_noise_img = add_sp_noise(img)


In [ ]:
printImgs([
    cv.blur(sp_noise_img,(3,3)),
    cv.blur(sp_noise_img,(5,5)),
    cv.blur(sp_noise_img,(7,7)),
    cv.blur(sp_noise_img,(9,9)),
])

In [ ]:
printImgs([
    cv.medianBlur(sp_noise_img, 3),
    cv.medianBlur(sp_noise_img, 5),
    cv.medianBlur(sp_noise_img, 7),
    cv.medianBlur(sp_noise_img, 9),
])

* Реализуйте медианный фильтр. Сравните с изображением, которое получается в результате применения функции cv2.medianBlur().

В этой реализации функция my_medianBlur принимает входное изображение и размер ядра ksize. Затем он вычисляет необходимый размер заполнения и создает дополненное изображение с помощью np.pad. Вложенный цикл используется для перебора каждого пикселя изображения и извлечения окрестности. Операция медианы применяется с помощью np.median, и полученное значение устанавливается в выходное изображение. Наконец, отфильтрованное изображение отображается с помощью cv2.imshow.


In [ ]:
plt.imshow(img)

In [ ]:
import numpy as np

def bad_my_medianBlur(image, ksize):
    # Get image dimensions
    height, width = image.shape[:2]
    ch = None
    if (len(image.shape) == 3):
        ch = image.shape[2]

    # Calculate padding size
    pad_size = ksize // 2
    
    # Create a padded image
    padded_image = np.pad(image, pad_size, mode='constant')
    
    # Create an empty output image
    filtered_image = np.zeros_like(image)
    
    # Apply median filter
    if ch:
        for c in range(ch):
            for i in range(height):
                for j in range(width):
                    # Extract neighborhood
                    neighborhood = padded_image[i:i+ksize, j:j+ksize, c].flatten()
                    
                    # Apply median operation
                    median_value = np.median(neighborhood)
                    
                    # Set filtered value in output image
                    filtered_image[i, j, c] = median_value
    
    return filtered_image

# Testing the custom bad_medianBlur function
bad_filtered_image = bad_my_medianBlur(img, 3)

# Display the filtered image
plt.imshow(bad_filtered_image)


In [ ]:
import numpy as np

def my_medianBlur(image, ksize):
    # Get image dimensions
    height, width = image.shape[:2]
    channels = None

    # if channels exists and image not grayscaled
    if (len(image.shape) == 3):
        channels = image.shape[2]

    # Calculate padding size
    pad_size = ksize // 2

    # if  image is color image
    if channels:
        # Create a padded image
        padded_image = np.pad(image, ((pad_size, pad_size), (pad_size, pad_size), (0, 0)), mode='constant')
        
        # Create an empty output image
        filtered_image = np.zeros_like(image)
        
        # Apply median filter
        for i in range(height):
            for j in range(width):
                # Extract neighborhood
                neighborhood = padded_image[i:i+ksize, j:j+ksize].reshape(-1, channels)
                
                # Apply median operation for each channel
                median_values = np.median(neighborhood, axis=0)
                
                # Set filtered values in output image
                filtered_image[i, j] = median_values

    # image is grayscaled
    else:
        # Create a padded image
        padded_image = np.pad(image, pad_size, mode='constant')
        
        # Create an empty output image
        filtered_image = np.zeros_like(image)
        
        # Apply median filter
        for i in range(height):
            for j in range(width):
                # Extract neighborhood
                neighborhood = padded_image[i:i+ksize, j:j+ksize].flatten()
                
                # Apply median operation
                median_value = np.median(neighborhood)
                
                # Set filtered value in output image
                filtered_image[i, j] = median_value
        
    return filtered_image


In [ ]:
# Testing the custom medianBlur function
filtered_image = my_medianBlur(img_gray, 3)

# Display the filtered image
plt.imshow(filtered_image, 'gray')

In [ ]:
# Testing the custom medianBlur function
filtered_image = my_medianBlur(img, 3)

# Display the filtered image
plt.imshow(filtered_image)


In [ ]:
# Testing the custom medianBlur function
sp_noise_filtered_image = my_medianBlur(sp_noise_img, 3)

# Display the filtered image
plt.imshow(sp_noise_filtered_image)

In [ ]:
printImgs([
    my_medianBlur(sp_noise_img, 3),
    my_medianBlur(sp_noise_img, 5),
    my_medianBlur(sp_noise_img, 7),
    my_medianBlur(sp_noise_img, 9),
])

In [ ]:
import numpy as np

def conv2D(img, kernel, padding=0, stride=1):
    img_height, img_width = img.shape[:2]
    kernel_height, kernel_width = kernel.shape
    
    # Calculate output dimensions based on padding and stride
    output_height = (img_height - kernel_height + 2 * padding) // stride + 1
    output_width = (img_width - kernel_width + 2 * padding) // stride + 1

    
    if (len(img.shape) == 3):
        # Apply padding to the input image
        padded_img = np.pad(img, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
        
        # Initialize the output image with zeros
        output = np.zeros((output_height, output_width, img.shape[2]))
        # Perform convolution operation for color image
        for y in range(0, output_height):
            for x in range(0, output_width):
                # Extract the region of interest from padded image
                for chanell in range(img.shape[2]):
                    roi = padded_img[y * stride:y * stride + kernel_height, x * stride:x * stride + kernel_width, chanell]

                    sm = np.sum(np.multiply(roi, kernel))
                    output[y, x, chanell] = sm

    # Perform convolution operation for grayscale
    else:
        # Apply padding to the input image
        padded_img = np.pad(img, ((padding, padding), (padding, padding)), mode='constant')
        
        # Initialize the output image with zeros
        output = np.zeros((output_height, output_width))
        for y in range(0, output_height):
            for x in range(0, output_width):
                roi = padded_img[y * stride:y * stride + kernel_height, x * stride:x * stride + kernel_width]

                sm = np.sum(np.multiply(roi, kernel))
                output[y, x] = sm

    return np.where(output<255,output,255).astype(np.uint8)

In [ ]:
# Testing conv2D function
kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
convolved_output = conv2D(img_gray, kernel, 1, 1)
plt.imshow(convolved_output, 'gray')

# Comparison with cv2.filter2D()
cv_output = cv.filter2D(img_gray, -1, kernel)
plt.imshow(cv_output, 'gray')

In [ ]:
ker = np.zeros((5,5),dtype=np.int8)
ker[0,:] = 1
ker[4,:] = -1

convolved_output = conv2D(img, ker, 1, 1)
plt.imshow(convolved_output)

In [ ]:
# Testing conv2D function
kernel = np.array([[1, 1, -1], [1, 1, -1], [1, -1, -1]])
convolved_output = conv2D(img, kernel, 1, 1)
plt.imshow(convolved_output)

In [ ]:
# Testing conv2D function
kernel = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
convolved_output = conv2D(img, kernel, 1, 1)
plt.imshow(convolved_output)

In [ ]:
# Comparison with cv2.filter2D()
kernel = np.array([[1, 1, -1], [1, 1, -1], [1, -1, -1]])
cv_output = cv.filter2D(img, -1, kernel)
plt.imshow(cv_output)